In [1]:
import os
import sys
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from typing import Type

from collections import namedtuple

In [15]:
class MandyocLayer:
    def __init__(self, rheology: type, density, interface, effective_viscosity_scale_factor=1.0, radiogenic_heat_production=0.0):
        """"
        rheology: class
            Rheological properties of the layer onbtained from LithologicalUnit class
        density: float
            Density of the layer [kg/m3]
        effective_viscosity_scale_factor: float
            Scale factor for the effective vistocisty
        radiogenic_heat_production: float
            Radiogenic heat production of the layer [W/kg]
        """

        self.rheology = LithologicalUnit(rheology)
        self.density = density
        self.interface = interface
        self.effective_viscosity_scale_factor = effective_viscosity_scale_factor
        self.radiogenic_heat_production = radiogenic_heat_production
        self.rheology_name = self.rheology.name
        self.pre_exponential_constant = self.rheology.pre_exponential_constant
        self.power_law_exponent = self.rheology.power_law_exponent
        self.activation_energy = self.rheology.activation_energy
        self.activation_volume = self.rheology.activation_volume
    
        # self.rheological_property = namedtuple(self.rheology, ['pre_exponential_constant',
        #                                                         'power_law_exponent',
        #                                                         'activation_energy',
        #                                                         'activation_volume'])
        
        # if(self.rheology == 'wet_quartz'):
        #     self.rheological_properties = self.rheological_property(8.574e-28, 4.0, 222.0e3, 0.0)
        # if(self.rheology == 'wet_olivine'):
        #     self.rheological_properties = self.rheological_property(1.393e-14, 3.0, 429.0e3, 15.0e-6)
        # if(self.rheology == 'dry_olivine'):
        #     self.rheological_properties = self.rheological_property(2.4168e-15, 3.5, 540.0e3, 25.0e-6)

class LithologicalUnit:
    """"
    This class calls the respective rheological properties of the given mineral

    mineral_name: class
        Mineral rheology written in CamelCase. For example, WetOlivine, DryOlivine, WetQuartz
    """
    def __init__(self, mineral_name: type):
        self.mineral_name = mineral_name() # mineral_name is a class, so we need to call it to get the object
        self.name = self.mineral_name.name
        self.pre_exponential_constant = self.mineral_name.pre_exponential_constant
        self.power_law_exponent = self.mineral_name.power_law_exponent
        self.activation_energy = self.mineral_name.activation_energy
        self.activation_volume = self.mineral_name.activation_volume

class WetOlivine:
    """
    Wet olivine rheological properties
    """
    def __init__(self):
        self.name = 'wet_olivine'
        self.pre_exponential_constant = 1.393e-14
        self.power_law_exponent = 3
        self.activation_energy = 429.0e3
        self.activation_volume = 15.0e-6

class DryOlivine:
    """
    Dry olivine rheological properties
    """
    def __init__(self):
        self.name = 'dry_olivine'
        self.pre_exponential_constant = 2.4168e-15
        self.power_law_exponent = 3.5
        self.activation_energy = 540.0e3
        self.activation_volume = 25.0e-6

class WetQuartz:
    """
    Wet quartz rheological properties
    """
    def __init__(self):
        self.name = 'wet_quartz'
        self.pre_exponential_constant = 8.574e-28
        self.power_law_exponent = 4.0
        self.activation_energy = 222.0e3
        self.activation_volume = 0.0

class Air:
    """
    Air rheological properties
    """
    def __init__(self):
        self.name = 'air'
        self.pre_exponential_constant = 1.0e-18
        self.power_law_exponent = 1.0
        self.activation_energy = 0.0
        self.activation_volume = 0.0
    

In [112]:
Nx = 1001
default_interface = np.ones(Nx) * 0

asthenosphere = MandyocLayer(WetOlivine, 3300.0, default_interface, 1.0, 7.38e-12)
lithospheric_mantle = MandyocLayer(DryOlivine, 3354.0, default_interface, 1.0, 9.0e-12)
lower_crust = MandyocLayer(WetQuartz, 2800.0, default_interface, 1.0, 0.8e-6 / 2800.0)
upper_crust = MandyocLayer(WetQuartz, 2800.0, default_interface, 1.0, 2.5e-6 / 2700.0)
air = MandyocLayer(Air, 1.0, default_interface, 1.0, 0.0)

stacked_layers = [asthenosphere, lithospheric_mantle, lower_crust, upper_crust, air] #bot to top

#Build layer_properties according to the order of the stack_layers
rheological_symbols = ['C', 'rho', 'H', 'A', 'n', 'Q', 'V']
rheological_properties = ['effective_viscosity_scale_factor', 'density', 'radiogenic_heat_production', 'pre_exponential_constant', 'power_law_exponent', 'activation_energy', 'activation_volume']

to_save = []
for symbol, prop in zip(rheological_symbols, rheological_properties):
    to_save.append(f"{symbol} {' '.join([str(layer.__dict__[prop]) for layer in stacked_layers])}")


for line in to_save:
    print(line)
# layer_properties = f"""
#                         C   {asthenosphere.effective_viscosity_scale_factor}   {lithospheric_mantle.effective_viscosity_scale_factor}   {lower_crust.effective_viscosity_scale_factor}   {upper_crust.effective_viscosity_scale_factor}   {air.effective_viscosity_scale_factor}
#                         rho {asthenosphere.density}                            {lithospheric_mantle.density}                            {lower_crust.density}                            {upper_crust.density}                            {air.density}
#                         H   {asthenosphere.radiogenic_heat_production}         {lithospheric_mantle.radiogenic_heat_production}         {lower_crust.radiogenic_heat_production}         {upper_crust.radiogenic_heat_production}         {air.radiogenic_heat_production}
#                         A   {asthenosphere.pre_exponential_constant}           {lithospheric_mantle.pre_exponential_constant}           {lower_crust.pre_exponential_constant}           {upper_crust.pre_exponential_constantcrust}      {air.pre_exponential_constant}
#                         n   {asthenosphere.power_law_exponent}                 {lithospheric_mantle.power_law_exponent}                 {lower_crust.power_law_exponent}                 {upper_crust.power_law_exponent}                 {air.power_law_exponent}
#                         Q   {asthenosphere.activation_energy}                  {lithospheric_mantle.activation_energy}                  {lower_crust.activation_energy}                  {upper_crust.activation_energy}                  {air.activation_energy} 
#                         V   {asthenosphere.activation_volume}                  {lithospheric_mantle.activation_volume}                  {lower_crust.activation_volume}                  {upper_crust.activation_volume}                  {air.activation_volume}
#                     """

C 1.0 1.0 1.0 1.0 1.0
rho 3300.0 3354.0 2800.0 2800.0 1.0
H 7.38e-12 9e-12 2.857142857142857e-10 9.25925925925926e-10 0.0
A 1.393e-14 2.4168e-15 8.574e-28 8.574e-28 1e-18
n 3 3.5 4.0 4.0 1.0
Q 429000.0 540000.0 222000.0 222000.0 0.0
V 1.5e-05 2.5e-05 0.0 0.0 0.0


In [13]:
rho_lc = 2800 #lower crust density [kg/m3]
C_lc = 1.0 #lower crust scaling factor
H_lc = 2.5e-6 / 2700.0 #lower crust radiogenic heat production [W/kg]

Nx = 1001
thickness_lower_crust = 40.0
lc_interface = np.ones(Nx) * thickness_lower_crust

lower_crust = MandyocLayer(WetOlivine, rho_lc, lc_interface, C_lc, H_lc)


array([40., 40., 40., 40., 40., 40., 40., 40., 40., 40., 40.])

8.574e-28

_tuplegetter(0, 'Alias for field number 0')

8.574e-28
4.0
